In [1]:
# Create preprocessed data table
# USE for Overviews
# Filmweb example data test
# XGBoost Regressor with GPU

In [2]:
import pandas as pd
import re
import os
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub

In [3]:
# os.environ["TFHUB_CACHE_DIR"] = r'C:\Users\jmay2\AppData\Local\Temp\tfhub_modules'
# module_url = "https://tfhub.dev/google/universal-sentence-encoder/3"
# embed = hub.Module(module_url)

In [6]:
embed = hub.load(r'C:\Users\jmay2\AppData\Local\Temp\tfhub_modules\42480c3c7f42bf87d36d4c58fc4374b08dae2909')

In [ ]:
# paragraph = (
#     "Universal Sentence Encoder embeddings also support short paragraphs. "
#     "There is no hard limit on how long the paragraph is. Roughly, the longer "
#     "the more 'diluted' the embedding will be.")

# message_embeddings = embed([paragraph])
# print(message_embeddings)

In [14]:
credits = pd.read_csv('./tmdb-movie-metadata/tmdb_5000_credits.csv')

In [15]:
movies = pd.read_csv('./tmdb-movie-metadata/tmdb_5000_movies.csv')

In [16]:
credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [17]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [18]:
movies_selected = movies[['id', 'budget', 'genres', 'keywords', 'overview', 'popularity', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'vote_average', 'vote_count']]

In [19]:
data_selected = pd.merge(credits, movies_selected, left_on='movie_id', right_on='id', how='left').drop('id', axis=1)

In [20]:
data_selected.head()

,movie_id,title,cast,crew,budget,genres,keywords,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,vote_average,vote_count
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",7.2,11800
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",6.9,4500
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",6.3,4466
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",7.6,9106
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",6.1,2124


In [21]:
def encode_overview(encoding_type=0):
    pass

In [22]:
name_regex = re.compile(r'\"name\": \"([a-zA-Z\s]+)\"')
director_regex = re.compile(r'\"job\": \"Director\", \"name\": \"([a-zA-Z\s]+)\"')
year_regex = re.compile(r'([0-9]{4})')

In [23]:
all_cast = set()
all_directors = set()
all_genres = set()
all_keywords = set()
all_companies = set()
all_countries = set()
all_languages = set()

In [24]:
# df['Date'] = df['Date'].apply(convert_to_year)
# my_series.apply(your_function, args=(2,3,4), extra_kw=1)

def prep_cast(data, name_regex, cast_member_number):
    cast_members = re.findall(name_regex, data)
    if len(cast_members) >= cast_member_number:
        cast_members = cast_members[:cast_member_number]
    for member in cast_members:
        all_cast.add(member)
    return cast_members
def prep_directors(data, director_regex):
    directors = re.findall(director_regex, data)
    for director in directors:
        all_directors.add(director)
    return directors
def prep_genres(data, name_regex):
    genres = re.findall(name_regex, data)
    for genre in genres:
        all_genres.add(genre)
    return genres
def prep_keywords(data, name_regex, keyword_number):
    keywords = re.findall(name_regex, data)
    if len(keywords) >= keyword_number:
        keywords = keywords[:keyword_number]
    for keyword in keywords:
        all_keywords.add(keyword)
    return keywords
def prep_companies(data, name_regex):
    companies = re.findall(name_regex, data)
    for company in companies:
        all_companies.add(company)
    return companies
def prep_countries(data, name_regex):
    countries = re.findall(name_regex, data)
    for country in countries:
        all_countries.add(country)
    return countries
def prep_languages(data, name_regex):
    languages = re.findall(name_regex, data)
    for language in languages:
        all_languages.add(language)
    return languages
def prep_year(data, year_regex):
    year = int(re.search(year_regex, data).group())
    return year

In [25]:
# def preprocess_columns(row, name_regex, director_regex, year_regex, cast_member_number, keyword_number):
#     cast_members = re.findall(name_regex, row['cast'])
#     if len(cast_members) >= cast_member_number:
#         cast_members = cast_members[:cast_member_number]
#     for member in cast_members:
#         all_cast.add(member)
#     directors = re.findall(director_regex, row['crew'])
#     for director in directors:
#         all_directors.add(director)
#     genres = re.findall(name_regex, row['genres'])
#     for genre in genres:
#         all_genres.add(genre)
#     keywords = re.findall(name_regex, row['keywords'])
#     if len(keywords) >= keyword_number:
#         keywords = keywords[:keyword_number]
#     for keyword in keywords:
#         all_keywords.add(keyword)
#     companies = re.findall(name_regex, row['production_companies'])
#     for company in companies:
#         all_companies.add(company)
#     countries = re.findall(name_regex, row['production_countries'])
#     for country in countries:
#         all_countries.add(country)
#     languages = re.findall(name_regex, row['spoken_languages'])
#     for language in languages:
#         all_languages.add(language)
#     year = int(re.search(year_regex, row['release_date']).group())
#     return cast_members, directors, genres, keywords, companies, countries, languages, year

In [26]:
# preprocess_columns(data_selected.iloc[0], name_regex, director_regex, year_regex, 5, 5)

In [ ]:
# message_embeddings = embed([data_selected.iloc[0]['overview']])
# print(message_embeddings)